## Kaggle competition scraper

this script scrapes the content of already stored competitions  
as they have a different formatting than datasets

In [1]:
# imports
import requests
import json
import os
import sys
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

#store_data(result[0]['html'], '../data/repositories/kaggle/notebook.html')
#store_data(result[0]['iframe'], '../data/repositories/kaggle/kernel.html')

In [3]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [24]:
# scrape links from notebooks.html

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
file_in = 'dataset.html'
url = 'https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/notebooks?sortBy=voteCount'

def scrape_dataset(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = {}
    
    partial = soup.find('div', {"id": "site-content"}) #class_="km-list km-list--avatar-list km-list--three-line")
    if partial == None:
        return []
    
    result['title'] = soup.find('h1', class_='competition-header__title').text.strip()
    result['subtitle'] = soup.find('h2', class_='competition-header__subtitle').text.strip()
    result['type'] = soup.find('p', class_='competition-header__classification-text').text.strip()
    try:
        result['organisation'] = soup.find('span', class_='competition-header__organization-name').text.strip()
    except:
        result['organisation'] = ''
    #result['teams'] = soup.find('', class_='').text.strip()
    result['description'] = soup.find('div', class_='markdown-converter__text--rendered').text.strip()
    #result['date'] = soup.find('', class_='').text.strip()
    try:
        result['reward'] = soup.find('div', class_='competition-header__prize').text.strip()
    except:
        result['reward'] = ''
    
    items = partial.find_all('li', class_='horizontal-list-item')
    #print(len(items))
    
    for i, item in enumerate(items):
        #print(i, item)
        if 'team' in item.text:
            result['teams'] = item.text.strip()
        if 'ago' in item.text:
            result['date'] = item.select('span>span')[0].get('title')
            result['date'] = result['date'].split(' (')
            result['date'] = result['date'][0]
        
        #print(link)
    
    return result

html = load_data(folder_base+folder+file_in)
if 'not found' in html:
    print(html)
links = scrape_dataset(html)
print(links)
#store_data(links, folder_base+folder+out, toJson=True)

{'title': 'Lyft 3D Object Detection for Autonomous Vehicles', 'subtitle': 'Can you advance the state of the art in 3D object detection?', 'type': 'Featured prediction Competition', 'organisation': 'Lyft', 'description': 'Self-driving technology presents a rare opportunity to improve the quality of life in many of our communities. Avoidable collisions, single-occupant commuters, and vehicle emissions are choking cities, while infrastructure strains under rapid urban growth. Autonomous vehicles are expected to redefine transportation and unlock a myriad of societal, environmental, and economic benefits. You can apply your data analysis skills in this competition to advance the state of self-driving technology.\nLyft, whose mission is to improve people’s lives with the world’s best transportation, is investing in the future of self-driving vehicles. Level 5, their self-driving division, is working on a fleet of autonomous vehicles, and currently has a team of 450+ across Palo Alto, London

In [26]:
# parse all competitions

path = '../data/repositories/kaggle/competitions/'
#path = '../data/repositories/kaggle/datasets/'
file = 'dataset.html'
#file_out = 'dataset.json'

LIMIT = 1
i = 0

folders = os.listdir(path)
for folder in folders:
    subfolders = os.listdir(os.path.join(path,folder))
    for subfolder in subfolders:
        i += 1
        item = os.path.join(path,folder,subfolder,file)
        print(item)
        if os.path.isfile(item):
            html = load_data(item)
            meta = scrape_dataset(html)
            print('meta', meta)
            output = item.replace('.html','.json')
            print('output', output)
            store_data(links, output, toJson=True)
        
        if i>=LIMIT:
            print('limit reached', i)
            sys.exit()

../data/repositories/kaggle/competitions/c\15-071x-the-analytics-edge-competition-spring-2015\dataset.html
meta {'title': '15.071x - The Analytics Edge (Spring 2015)', 'subtitle': 'Test your analytics skills by predicting which New York Times blog articles will be the most popular', 'type': 'Research prediction Competition', 'organisation': '', 'description': 'IMPORTANT NOTE: This competition is only open to students of the MITx free, online course 15.071x - The Analytics Edge.\nWhat makes online news articles popular?\nNewspapers and online news aggregators like Google News need to understand which news articles will be the most popular, so that they can prioritize the order in which stories appear. In this competition, you will predict the popularity of a set of New York Times blog articles from the time period September 2014-December 2014.\nThe following screenshot shows an example of the New York Times technology blog "Bits" homepage:\n\nMany blog articles are published each day, a

SystemExit: 